# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [29]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [31]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# Setup connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [33]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS student_udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('student_udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [35]:
## Show artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Define query from business unit and for table generation
query_businessUnit = "SELECT artist, songTitle, length from song_library WHERE sessionId = 338 and itemInSession = 4"
query_tableCreation = "CREATE TABLE IF NOT EXISTS song_library"

# Create table specifically for query
# To uniquely identify a record in the table the session Id and item in the session need to be combined 
query = query_tableCreation + "(sessionID int, iteminSession int, artist text, songTitle text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [36]:
## Access datasource (csv) and load records into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: # iterate through records in csv
        query = "INSERT INTO song_library (sessionID, iteminSession, artist, songTitle, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
## Execute query of the business unit. Show rows to validate result.
try:
    rows = session.execute(query_businessUnit)
except Exception as e:
        print(e)
    
for row in rows:
    print (row.artist)
    print (row.songtitle)
    print (row.length)

Faithless
Music Matters (Mark Knight Dub)
495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [38]:
## Show name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Define query from business unit and for table generation
query_businessUnit1 = "SELECT iteminsession, artist, songTitle, firstName, lastName from user_songs \
                       WHERE userId = 10 AND sessionId = 182"
query_tableCreation1 = "CREATE TABLE IF NOT EXISTS user_songs"

# Create table specifically for query, ensure that songs being sorted by items in session
query1 = query_tableCreation1 + "(userId int, sessionId int, itemInSession varchar, artist text, songTitle text, \
                                 firstName text, lastName text, PRIMARY KEY ((userId,  sessionId), itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
    
## Access datasource (csv) and load records into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO user_songs (userId, sessionID, itemInSession, artist, songTitle, firstName, lastName)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query1, (int(line[10]), int(line[8]), line[3], line[0], line[9], line[1], line[4]))

## Execute business query and validate data manually via print
try:
    rows = session.execute(query_businessUnit1)
except Exception as e:
        print(e)

for row in rows:
    print (row.iteminsession, row.artist, row.songtitle, row.firstname, row.lastname)       

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [55]:
## Show every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'

# Define query from business unit and for table generation
query_businessUnit2 = "SELECT songTitle, firstName, lastName from user_behaviour \
                       WHERE songTitle = 'All Hands Against His Own'"
query_tableCreation2 = "CREATE TABLE IF NOT EXISTS user_behaviour"

# Create table specifically for query
query_tableUserCreation = query_tableCreation2 + "(songTitle text, firstName text, lastName text, \
                                                   PRIMARY KEY (songTitle))"
try:
    session.execute(query_tableUserCreation)
except Exception as e:
    print(e)
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_musicUser_Insert = "INSERT INTO user_behaviour (songTitle, firstName, lastName)"
        query_musicUser_Insert = query_musicUser_Insert + " VALUES (%s, %s, %s)"
        session.execute(query_musicUser_Insert, (line[9], line[1], line[4]))

## Execute business query  and validate data manually via print
try:
    rows = session.execute(query_businessUnit2)
except Exception as e:
        print(e)

for row in rows:
   print(row.songtitle, row.firstname, row.lastname)          

All Hands Against His Own Tegan Levine


### Drop the tables before closing out the sessions

In [56]:
## Ensure tables being dropped to avoid issues when tables being recreated e.g. for a new session
def drop_tables(query):
    try:
        session.execute(query)
    except Exception as e:
        print(e)  

# Define tables being dropped
query_drop = "DROP TABLE IF EXISTS "
queries = [query_drop + "song_library", query_drop + "user_songs", \
           query_drop + "user_behaviour"]

# Iterate over tables and execute table drops
for query in queries:
    drop_tables(query)
    print("Confirmed: " + query)

Confirmed: DROP TABLE IF EXISTS song_library
Confirmed: DROP TABLE IF EXISTS user_songs
Confirmed: DROP TABLE IF EXISTS user_behaviour


### Close the session and cluster connection¶

In [27]:
# Properly close session
session.shutdown()
cluster.shutdown()